In [ ]:
import gmsh
import sys
gmsh.initialize()
gmsh.model.add("mesh_with_occt")
# Define geometry parameters (example: two spheres)
innerRadius = 5
outerRadius = 10
# Create the outer sphere (radius 10) and inner sphere (radius 5)
outerSphere = gmsh.model.occ.addSphere(0, 0, 0, outerRadius)
innerSphere = gmsh.model.occ.addSphere(0, 0, 0, innerRadius)
# Perform a Boolean difference to create a shell, but keep the inner sphere (tool)
cutResult = gmsh.model.occ.cut([(3, outerSphere)], [(3, innerSphere)],
removeTool=False)
# For safety, print the result to inspect the structure.
print("Cut result:", cutResult)
# Synchronize the CAD kernel with the Gmsh model
gmsh.model.occ.synchronize()
# Identify the volumes: we expect two volumes (the inner sphere and the shell)
volEntities = gmsh.model.getEntities(2)
print("Volumes:", volEntities)
shell_volume = None
for (dim, tag) in volEntities:
if tag != innerSphere:
shell_volume = tag
break
if shell_volume is None:
raise RuntimeError("Could not identify the shell volume.")
# Define physical groups for the inner sphere and shell
gmsh.model.addPhysicalGroup(3, [innerSphere], tag=1)
gmsh.model.setPhysicalName(3, 1, "Inner")
gmsh.model.addPhysicalGroup(3, [shell_volume], tag=2)
gmsh.model.setPhysicalName(3, 2, "Shell")
# Create a MathEval field to control the mesh size based on z.
fieldTag = gmsh.model.mesh.field.add("MathEval")
# gmsh.model.mesh.field.setString(fieldTag, "F", "if(z-1e-4<0,0.9,2.3)")
gmsh.model.mesh.field.setString(fieldTag, "F", "0.95")
# Use this field as the background mesh field.
gmsh.model.mesh.field.setAsBackgroundMesh(fieldTag)
# Generate the 3D mesh
gmsh.model.mesh.generate(3)
# Save mesh in VTK format
gmsh.write("vtk_meshes/two_spheres.vtk")
if "-nopopup" not in sys.argv:
gmsh.fltk.run()
gmsh.finalize()